In [ ]:
import numpy as np
import gym
import matplotlib.pyplot as plt
from f110_gym.envs.base_classes import Integrator
import random
import os
import pandas as pd
import time
from sklearn.preprocessing import normalize
from scipy.sparse import csr_matrix
import sys
from f110_gym.envs.f110_env import F110Env
import logging

logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    filename='SARSA_logs.log',
    filemode='a'
)

class Reward:
    def __init__(self, min_speed=0.8, max_speed=2, map_centers=None,track_width = 2.2):
        self.min_speed = min_speed
        self.max_speed = max_speed
        self.set_parameters(map_centers,track_width)
        
        # Hyperparameters
        self.epsilon = 1e-5
        self.distance_travelled = 0

        # Centering reward function
        self.func = lambda y : 2 * (np.exp(-0.017*y) - 0.5)
    
    def set_parameters(self,map_centers, track_width):
        self.map_centers = map_centers
        # Initial point and center that detrmines the position at the start of episode
        self.initial_point = np.array([[0, 0]])
        self.initial_center_idx , _ = self.__calculate_distance_from_center(self.map_centers,self.initial_point)
        self.initial_center = self.map_centers[self.initial_center_idx]

        # Race Track parameters
        self.distance_between_centers = np.hstack([[0.],np.linalg.norm(self.map_centers[:-1,:]- self.map_centers[1:,:],axis=1)])
        self.total_track_length = np.sum(self.distance_between_centers)
        self.track_width = track_width


    
    def __calculate_distance_from_center(self, centers,curr):
        distances = np.linalg.norm(centers - curr, axis=1)
        idx = np.argmin(distances)
        return idx, distances[idx]
    
    def reset(self, point):
        self.distance_travelled = 0
        self.initial_point = point
        idx , _ = self.__calculate_distance_from_center(self.map_centers,self.initial_point)
        self.initial_center_idx = idx
        self.initial_center = self.map_centers[self.initial_center_idx]

    def exponential_angle(self, angle):
        if angle <=90:
            return self.func(angle)
        return -1
    
    def progress_reward(self, curr_position, next_position):
        distance = np.linalg.norm(curr_position - next_position)
        self.distance_travelled += distance
        return self.distance_travelled / self.total_track_length

    def centering_reward(self, curr_position,next_position):
        
        position_vector = next_position - curr_position

        curr_idx, c = self.__calculate_distance_from_center(self.map_centers,curr_position)
        # print(f'Curr position: {curr_position}, Next position: {next_position} with initial center idx: {curr_idx}')

        if curr_idx == self.map_centers.shape[0] - 1:
            indices = [curr_idx-1,0]
        elif curr_idx == 0:
            indices = [self.map_centers.shape[0]-1,curr_idx+1]
        else:
            indices = [curr_idx-1, curr_idx+1]
            
        # print(f'Indices: {indices}')
        # print(np.vstack([self.map_centers[indices[0],:],self.map_centers[indices[1],:]]))

        next_idx, n = self.__calculate_distance_from_center(np.vstack([self.map_centers[indices[0],:],self.map_centers[indices[1],:]]),next_position)
        next_idx = indices[next_idx]
        # print(f'Next index is {next_idx} and distance is {n}')

        curr_center = self.map_centers[curr_idx]
        next_center = self.map_centers[next_idx]

        if curr_idx == next_idx:
            return -1
        
        # print(f'Current center: {curr_center}, Next center: {next_center}')
        center_vector = next_center - curr_center
        
        dot_product = np.dot(center_vector, position_vector)
        norm_product = np.linalg.norm(center_vector) * np.linalg.norm(position_vector)
        cosine_angle = dot_product / norm_product
        angle_rad = np.arccos(np.clip(cosine_angle, -1.0, 1.0))

        # print(f'Center vector: {center_vector} and Position vector: {position_vector}')

        return self.exponential_angle(np.degrees(angle_rad))
    
    def milestone_reward(self, next_position):
        idx, _ = self.__calculate_distance_from_center(self.map_centers,next_position)
        if idx!=self.initial_center_idx:
            travelled = np.linalg.norm(self.initial_center - next_position) / self.total_track_length
            if  travelled >= np.abs(self.distance_between_centers[idx] - self.distance_between_centers[self.initial_center_idx]) / self.total_track_length :
                self.initial_center_idx = idx
                self.initial_center = self.map_centers[idx]
                return 5
        return 0
    
    def calculate_reward(self, curr_position, next_position):
        progress_reward = self.progress_reward(curr_position, next_position)
        centering_reward = self.centering_reward(curr_position, next_position)
        milestone_reward = self.milestone_reward(next_position)
        # print(f"Distance reward: {progress_reward}, Centering reward: {centering_reward}, Milestone reward: {milestone_reward}")
        return progress_reward + centering_reward + milestone_reward

In [ ]:
class Reward:
    def __init__(self, min_speed=0.8, max_speed=2,num_speeds=5 ,map_centers=None,track_width = 2.2,logger = None):
        # Keep existing parameters
        self.min_speed = min_speed
        self.max_speed = max_speed
        self.num_speeds = num_speeds
        self.speeds = np.linspace(min_speed, max_speed, num_speeds)
        self.mean_speed = np.mean(self.speeds)
        self.std_speed = np.std(self.speeds)
        self.set_parameters(map_centers,track_width)
        
        # Hyperparameters
        self.epsilon = 1e-5
        self.distance_travelled = 0
        self.milestone = 0
        self.centerline_scale = 5
        self.progress_scale = 1.0
        self.logger = logger

    def set_parameters(self,map_centers, track_width):
        '''
        Helper function to set the parameters of the reward function externally from the class instance
        Args:
            map_centers (np.ndarray): Array of map centers.
            track_width (float): Width of the track.
        '''
        self.map_centers = map_centers
        # Initial point and center that determines the position at the start of episode
        self.initial_point = np.array([[0, 0]])
        self.initial_center_idx , _ = self.__calculate_distance_from_center(self.map_centers,self.initial_point)
        self.initial_center = self.map_centers[self.initial_center_idx]

        # Race Track parameters
        self.distance_between_centers = np.hstack([[0.],np.linalg.norm(self.map_centers[:-1,:]- self.map_centers[1:,:],axis=1)])
        self.total_track_length = np.sum(self.distance_between_centers)
        self.track_width = track_width
    
    def __calculate_distance_from_center(self, centers,curr):
        '''
        Helper function to calculate the distance from the all centers of the track to the current position
        Args:
            centers (np.ndarray): Array of map centers.
            curr (np.ndarray): Current position of the agent.
        Returns:
            idx (int): Index of the closest center.
            distance (float): Distance to the closest center.
        '''
        distances = np.linalg.norm(centers - curr, axis=1)
        idx = np.argmin(distances)
        return idx, distances[idx]
    
    def reset(self, pos):
        """
        Reset the reward function state, supporting arbitrary starting position and heading.
        
        Args:
            pos (np.ndarray): Starting position of the agent.
            heading (float, optional): Starting heading of the agent in radians.
        """
        self.distance_travelled = 0
        self.milestone = 0
        self.initial_point = pos
        
        # Initialize starting center reference
        self.initial_center_idx, _ = self.__calculate_distance_from_center(self.map_centers, self.initial_point)
        self.initial_center = self.map_centers[self.initial_center_idx]
    
    def centerline_reward(self, curr_pos, next_pos, curr_center_idx, next_center_idx, curr_dist,next_dist ,threshold_angle=0.2, threshold_dist = 0.35):
        """
        Calculate the centerline reward based on the distance from the centerline. 
        Angles are computred in radians.
        Restricting the large negative reward to -1000 and positive to 100
        
        Args:
            pos (np.ndarray): Current position of the agent.
        
        Returns:
            float: Centerline reward.
        """
        
        if curr_center_idx == next_center_idx:
            next_center_idx = (next_center_idx + 1) % len(self.map_centers)
        
        movement_vector = next_pos - curr_pos
        movement_vector /= (np.linalg.norm(movement_vector) + self.epsilon)

        centerline_vector = self.map_centers[next_center_idx] - self.map_centers[curr_center_idx]
        centerline_vector /= (np.linalg.norm(centerline_vector) + self.epsilon)

        angle = np.arctan2(centerline_vector[1], centerline_vector[0]) - np.arctan2(movement_vector[1], movement_vector[0])
        angle = np.arctan2(np.sin(angle), np.cos(angle))  # Normalize angle to [-pi, pi]

        angle = abs(angle)  # Consider only the absolute angle deviation

        if angle <= threshold_angle:
            reward = 2.0 - (angle / threshold_angle)
        else:
            # Exponential decay
            reward = -4.3 * np.exp(3 * (angle - threshold_angle))

        # print(f'Angle: {angle}, Angle Reward: {reward}')
        
        if next_dist <= threshold_dist:
            # Reward increases as distance gets closer to 0
            dist_reward = 2.0 * (1 - next_dist / threshold_dist)
        else:
            # Penalize exponentially as distance increases beyond threshold
            dist_reward = -2.0 * np.exp(1.5 * (next_dist - threshold_dist))
        
        # print(f'Next Distance: {next_dist},Distance Reward: {dist_reward}')
        # Combine rewards
        total_reward = reward + dist_reward
        
        # You can also add a bonus if the agent is improving its distance to centerline
        if next_dist < curr_dist:
            total_reward += 1  # Small bonus for getting closer to centerline

        return np.clip(total_reward,-1000,100)

    def speed_reward(self, speed):
        """
        Calculate the speed reward based on the speed of the agent.
        Uses gaussian distribution of the speeds and rewrads agent agent positively for speeds that fall in the center of the distribution and negative on both sides.
        
        Args:
            speed (float): Speed of the agent.
        
        Returns:
            float: Speed reward.
        """
        # Calculate the Gaussian probability density function
        # print(f'Chosen Speed is {speed}')
        reward =  4 * ((1/np.sqrt(2*np.pi*self.std_speed**2 ) * np.exp(-0.5/(self.std_speed**2) * (speed-self.mean_speed)**2)) - self.mean_speed/self.speeds[-1])
        return reward
        
    
    def progress_reward(self, curr_pos, next_pos, curr_center_idx, next_center_idx):
        """
        Calculate the progress reward based on the distance travelled along the track. 
        Restricting the positive reward to +100
        
        Args:
            curr_pos (np.ndarray): Current position of the agent.
            next_pos (np.ndarray): Next position of the agent.
            curr_center_idx (int): Index of the current center.
            next_center_idx (int): Index of the next center.
        
        Returns:
            float: Progress reward.
        """
        if curr_center_idx == next_center_idx:
            next_center_idx = (next_center_idx + 1) % len(self.map_centers)
        
        distance = np.linalg.norm(next_pos - curr_pos)
        self.distance_travelled += distance

        reward = self.distance_travelled
        if self.distance_travelled > 100:
            self.distance_travelled = 0
        
        return reward
    
    def calculate_reward(self,curr_pos,next_pos,speed):
        """
        Calculate the total reward based on centerline and progress rewards.
        
        Args:
            curr_pos (np.ndarray): Current position of the agent.
            next_pos (np.ndarray): Next position of the agent.
        
        Returns:
            float: Total reward.
        """
        curr_center_idx, curr_dist = self.__calculate_distance_from_center(self.map_centers, curr_pos)
        next_center_idx, next_dist = self.__calculate_distance_from_center(self.map_centers, next_pos)

        centerline_reward = self.centerline_reward(curr_pos, next_pos, curr_center_idx, next_center_idx, curr_dist, next_dist)
        # progress_reward = self.progress_reward(curr_pos, next_pos, curr_center_idx, next_center_idx)
        speed_reward = self.speed_reward(speed)
        
        total_reward = centerline_reward + speed_reward#+ progress_reward

        # print(f'Centerline Reward: {centerline_reward}, Speed reward: {speed_reward} \n Total Reward: {total_reward}')  # Progress Reward: {progress_reward}, \n Total Reward: {total_reward}

        return total_reward

In [ ]:
class IndexSelector:
    def __init__(self, num_indices):
        self.set_parameters(num_indices)
    
    def set_parameters(self, num_indices):
        self.num_indices = num_indices
        self.visited_indices = set()
        self.probabilities = np.ones(num_indices) / num_indices
        self.all_indices = np.arange(self.num_indices)
    
    def select_index(self):
        if len(self.visited_indices) == self.num_indices:
            # Reset the probabilities and visited indices
            print('Visited all indices, resetting')
            self.visited_indices = set()
            self.probabilities = np.ones(self.num_indices) / self.num_indices

        # Select an index based on the current probabilities
        random_idx = np.random.choice(self.all_indices, p=self.probabilities)

        # Update the probabilities
        self.visited_indices.add(random_idx)
        if len(self.visited_indices) < self.num_indices:
            self.probabilities[random_idx] = 0
            remaining_prob = 1 - np.sum(self.probabilities)
            self.probabilities[self.probabilities > 0] += remaining_prob / np.sum(self.probabilities > 0)

        return random_idx

In [ ]:
class F1Tenth_navigation:

    def __init__(self,gym_env_code='f110_gym:f110-v0', num_agents=1, map_path=['./f1tenth_racetracks/Austin/Austin_map'], map_ext='.png', sx=0., sy=0., stheta=0., map_centers_file=None, save_path=None, track_name=None, inference=None,reward_file=None,collision_file=None):

        # Environment setup
        self.path_counter = 0
        self.sx, self.sy, self.stheta = sx, sy, stheta
        self.save_path = save_path
        self.track_name = track_name
        self.num_agents = num_agents
        self.map_path = map_path
        self.map_ext = map_ext
        self.map_centers_file = map_centers_file

        self.logger = logging.getLogger(__name__)
        self.logger.info(f'Initializing F1Tenth_navigation with map: {self.map_path[self.path_counter]}')
    
        self.env = gym.make(gym_env_code, map=self.map_path[self.path_counter], map_ext=self.map_ext, num_agents=self.num_agents, timestep=0.01, integrator=Integrator.RK4)
        self.env.add_render_callback(self.render_callback)
       
        file = pd.read_csv(self.map_centers_file[self.path_counter])
        file.columns = ['x', 'y', 'w_r', 'w_l']
        file.index = file.index.astype(int)
        self.map_centers = file.values[:, :2]
        self.track_headings = self.calculate_track_headings(self.map_centers)
        self.track_width = file.loc[0,'w_r'] + file.loc[0,'w_l']
        self.reward_file = reward_file
        self.collision_file = collision_file
        self.track_headings = self.calculate_track_headings(self.map_centers)
        self.track_center_counter = self.map_centers.shape[0]  # Number of centers in the track


        # Random Seed
        self.random_seed = 42
        np.random.seed(self.random_seed)

        # Environment Observation Parameters
        self.num_beams = 1080
        self.n_features = 11
        self.angle = 220

        # LiDAR downsampling parameters
        self.n_sectors = 22
        self.normalized_lidar = np.zeros((1,self.n_sectors))

        # Action Space Parameters
        self.num_angles = self.n_sectors
        self.num_speeds = 5


        # State Space Parameters
        self.num_states = 2 ** self.n_features

        # Speed Parameters
        self.min_speed = 0.8
        self.max_speed = 1.8

        # Action Space
        self.angles_deg = np.linspace(-self.angle // 2, self.angle // 2, self.num_angles)[::-1]
        self.angles = np.radians(self.angles_deg)
        self.speeds = np.linspace(self.min_speed, self.max_speed, self.num_speeds)
    
        # State Space - Q-Table
        if inference is not None:
            self.weights = np.load(inference)
            self.num_collisions = int(inference.split('_')[-1].split('.')[0])
            print(f'Loaded Weights')
        else:
            # self.weights = np.zeros((self.num_states,self.num_angles,self.num_speeds))
            self.weights = np.random.randn(self.num_states,self.num_angles,self.num_speeds)
            self.num_collisions = 0
        
        self.max_weight = 5

        # ELigibility Trace
        self.ET_IS = np.zeros((self.num_states,self.num_angles,self.num_speeds))

        # projection matrix
        if self.n_features == 10:
            zero_prob = 0.85
            one_prob = 0.15
        if self.n_features == 11:
            zero_prob = 0.8
            one_prob = 0.2

        # zero_prob = 0.5
        # one_prob = 0.5
        
        self.projection_matrix = self.get_projection_matrix(zero_prob=zero_prob,one_prob=one_prob)
        # self.bias = np.linspace(-1,1,self.n_features).reshape(1,-1)
        self.bias = np.zeros((1,self.n_features))

        # binary powers
        self.binary_powers = np.array([2 ** i for i in range(self.n_features)])

        # Training Variables
        self.curr_state = None
        self.next_state = None
        
        self.action_threshold_decay = 0.9998
        self.action_threshold = 0.1 * (self.action_threshold_decay ** self.num_collisions)

        # Imported Classes
        self.reward_class = Reward(min_speed=self.min_speed, max_speed=self.max_speed, num_speeds=self.num_speeds,map_centers=self.map_centers, track_width=self.track_width,logger = self.logger)
        self.index_selector = IndexSelector(self.map_centers.shape[0])      

        # SARSA Parameters
        self.learning_rate = 1e-3
        self.discount_factor = 0.95
        self.decay_rate = 0.9

        # Reward
        self.reward = 0
        self.episode_reward = 0
        self.cumulative_reward = 0
        self.episodic_rewards = [0]

        # Time
        self.collision_times = [0]       


    def calculate_track_headings(self,track_centers, window_size=5):
        """
        Calculates orientations for track traversal.
        
        Args:
            track_centers (np.ndarray): Shape (N, 2) array of track center points (x, y)
            window_size (int): Number of points to consider for smoothing
        
        Returns:
            np.ndarray: Shape (N,) array of orientation angles in radians
        """
        num_points = track_centers.shape[0]
        half_window = window_size // 2
        
        # Create indices for the future points (with wraparound)
        future_indices = (np.arange(num_points) + half_window) % num_points
        
        # Get the future points
        future_points = track_centers[future_indices]
        
        # Calculate direction vectors
        direction_vectors = future_points - track_centers
        
        # Calculate angles using arctan2
        orientations = np.arctan2(direction_vectors[:, 1], direction_vectors[:, 0])
        
        return orientations

    def __update_map(self):
        '''
        Update the map of the environment to the next map in the list.
        This function is called after fixed no. of collision with the environment, usually set to 2000 in the training loop.

        '''
        if self.env.renderer is not None:
            self.env.renderer.close()
        self.path_counter += 1
        if self.path_counter == len(self.map_path):
            self.path_counter = 0
        self.env.map_name = self.map_path[self.path_counter]
        self.env.update_map(f'{self.map_path[self.path_counter]}.yaml',self.map_ext)
        F110Env.renderer = None
        file = pd.read_csv(self.map_centers_file[self.path_counter])
        file.columns = ['x', 'y', 'w_r', 'w_l']
        file.index = file.index.astype(int)
        self.map_centers = file.values[:, :2]
        self.track_width = file.loc[0,'w_r'] + file.loc[0,'w_l']
        self.track_headings = self.calculate_track_headings(self.map_centers)
        self.track_center_counter = self.map_centers.shape[0]  # Reset the counter to the number of centers
        print(f'Map updated to {self.track_name[self.path_counter]}')
        self.logger.info('-------'*20)
        self.logger.info(f'Initializing F1Tenth_navigation with map: {self.map_path[self.path_counter]}')
        
        
    def render_callback(self, env_renderer):
        e = env_renderer
        x = e.cars[0].vertices[::2]
        y = e.cars[0].vertices[1::2]
        top, bottom, left, right = max(y), min(y), min(x), max(x)
        e.score_label.x = left
        e.score_label.y = top - 700
        e.left = left - 800
        e.right = right + 800
        e.top = top + 800
        e.bottom = bottom - 800
    
    def calculate_track_headings(self,track_centers, window_size=5):
        """
        Calculates orientations for track traversal.
        
        Args:
            track_centers (np.ndarray): Shape (N, 2) array of track center points (x, y)
            window_size (int): Number of points to consider for smoothing
        
        Returns:
            np.ndarray: Shape (N,) array of orientation angles in radians
        """
        num_points = track_centers.shape[0]
        half_window = window_size // 2
        
        # Create indices for the future points (with wraparound)
        future_indices = (np.arange(num_points) + half_window) % num_points
        
        # Get the future points
        future_points = track_centers[future_indices]
        
        # Calculate direction vectors
        direction_vectors = future_points - track_centers
        
        # Calculate angles using arctan2
        orientations = np.arctan2(direction_vectors[:, 1], direction_vectors[:, 0])
        
        return orientations


    def get_statistical_properties(self,lidar_input,n_sectors=None):
        '''
        Function that is used to downsample the Lidar input.
        This function takes the Lidar input and divides it into n_sectors number of sectors.
        It then calculates the median of each sector and returns the median values.
        Args:
            lidar_input (np.ndarray): Lidar input.
            n_sectors (int): Number of sectors to downsample the Lidar input into.
        '''
        assert n_sectors is not None, "Number of sectors must be provided"
        #  The [100 :-100] is for selecting only those rays corresponding to 220 fov.
        sector_size = np.asarray(lidar_input[100:-100],dtype=np.float32).shape[0] // n_sectors
        sectors = lidar_input[:sector_size * n_sectors].reshape(n_sectors, sector_size)
        return np.median(sectors, axis=1).reshape(1,-1)
    
    def binarize_vector(self,vector):
        '''
        Function that is used to binarize the input.
        This function takes the projected downsampled-Lidar and binarizes it based on the threshold.
        Args:
            vector (np.ndarray): Projected downsampled-Lidar input.
        Returns:
            np.ndarray: Binary represnetation of the Lidar data which is used as a state.
        '''

        threshold = (np.min(vector)+ np.max(vector))/2
        return np.where(vector > threshold, 1, 0)
        # return np.where(vector > 0, 1, 0)

    def get_projection_matrix(self,zero_prob=0.5,one_prob=0.5):
        '''
        Function that is used to generate the projection matrix.
        This function takes the number of features and the number of angles and generates a random projection matrix.
        This function is called only once to generate the projection matrix and saves it to a file.
        Args:
            zero_prob (float): Probability of selecting 0.
            one_prob (float): Probability of selecting 1.
        Returns:
            np.ndarray: Projection matrix.
        '''
        # Generate a random matrix with values 0 and 1 based on the given probabilities [prob_0,prob_1]
        if not os.path.exists('Projection_matrices'):
            os.mkdir('Projection_matrices')
        if not os.path.exists(os.path.join('Projection_matrices', f'projection_{self.n_features}f_{self.num_angles}a_s{self.random_seed}.npy')):
            # std = np.sqrt(1/self.n_features)
            # matrix = np.random.choice([-1/std, 1/std], size=(self.n_sectors,self.n_features),p=[zero_prob, one_prob])
            matrix = np.random.choice([0, 1], size=(self.n_sectors, self.n_features), p=[zero_prob,one_prob])
            # matrix = np.random.normal(loc=0.0, scale=1/std, size=(self.n_sectors, self.n_features))
            np.save(os.path.join('Projection_matrices', f'projection_{self.n_features}f_{self.num_angles}a_s{self.random_seed}.npy'), matrix)
        else:
            matrix = np.load(os.path.join('Projection_matrices', f'projection_{self.n_features}f_{self.num_angles}a_s{self.random_seed}.npy'))
        return matrix

    def get_binary_representation(self,lidar_input):
        '''
        Function that is used to get the binary representation of the Lidar input.
        This function takes the Lidar input and projects it using the projection matrix.
        It then binarizes the projected Lidar input and returns the binary representation.
        the bias used is here is some types of non linear projections. It is set to zero here.
        Args:
            lidar_input (np.ndarray): Lidar input.
        Returns:
            np.ndarray: Binary representation of the Lidar input.
        '''
        self.normalized_lidar = normalize(lidar_input,axis=1)
        # Do not normalize, just use the raw data
        return self.binarize_vector(np.dot(lidar_input,self.projection_matrix) + self.bias)
    

    def get_state(self, binary):
        return np.dot(binary[0], self.binary_powers)
    

    def select_action(self, state):
        '''
        Function that is used to select the action of the agent.
        This function takes the state of the agent and selects an action based on the Weight matrix.
        The action is selected based on the epsilon greedy policy.
        Args:
            state (int): State of the agent.
        Returns:
            tuple: Angle index and speed index of the selected action.
        '''
        random_number = np.random.rand()
        if random_number < self.action_threshold:
            angle_index = np.random.randint(0, self.num_angles)
            speed_index = np.random.randint(0, self.num_speeds)
        else:
            max_value = np.max(self.weights[state])
            max_indices = np.argwhere(self.weights[state] == max_value)
            angle_index, speed_index  = max_indices[np.random.choice(np.arange(len(max_indices)))]
        
        return angle_index, speed_index

    def select_action_inference(self, state):
        '''
        Function that is used to select the action of the agent during inference.
        This function takes the state of the agent and selects an action based on the Weight matrix.
        Args:
            state (int): State of the agent.
        Returns:
            tuple: Angle index and speed index of the selected action.
        '''
        max_indices = np.argwhere(self.weights[state] == np.max(self.weights[state]))
        angle_index, speed_index  = max_indices[np.random.choice(np.arange(len(max_indices)))]
        return angle_index, speed_index

    def sarsa_weight_update(self,angle_idx,speed_idx,reward):
        next_angle_idx,next_speed_idx = self.select_action(self.next_state)
        delta = reward + self.discount_factor * self.weights[self.next_state,next_angle_idx,next_speed_idx] - self.weights[self.curr_state,angle_idx,speed_idx]

        self.weights += self.learning_rate * delta * self.ET_IS
        return next_angle_idx,next_speed_idx

    def set_eligibility_traces(self,angle_idx,speed_idx):
        self.ET_IS [self.curr_state,angle_idx,speed_idx] = 1

    def decay_eligibility_traces(self):
        self.ET_IS *= self.discount_factor * self.decay_rate

    def save_reward_time(self):
        if not os.path.exists(os.path.join(self.save_path)):
            os.mkdir(os.path.join(self.save_path))
        
        if self.reward_file is not None:
            r = np.append(np.load(self.reward_file), self.episodic_rewards)
            t = np.append(np.load(self.collision_file), self.collision_times)
            np.save(os.path.join(self.save_path, f'rewards.npy'), np.array(r))
            np.save(os.path.join(self.save_path, f'times.npy'), np.array(t))
        else:
            np.save(os.path.join(self.save_path, f'rewards.npy'), np.array(self.episodic_rewards))
            np.save(os.path.join(self.save_path, f'times.npy'), np.array(self.collision_times))

    def save_weights(self):
        if not os.path.exists(os.path.join(self.save_path)):
            os.mkdir(os.path.join(self.save_path))
        np.save(os.path.join(self.save_path, f'{self.track_name[self.path_counter]}_{self.num_collisions + 1}.npy'), self.weights)
        # print(f'File saved')


    def train(self):
        
        obs, step_reward, done, info = self.env.reset(np.array([[self.sx, self.sy, self.track_headings[0]]]))
        lidar = obs['scans'][0]
        lidar_down_sampled = self.get_statistical_properties(lidar,n_sectors=self.n_sectors)
        self.curr_state = self.get_state(self.get_binary_representation(lidar_down_sampled))
        self.reward_class.reset(np.array([[self.sx, self.sy]]))
        angle_index,speed_index = self.select_action(self.curr_state)
        start_time = time.time()
        while True:
            # np.save(f'./LiDAR_scans/scan_{self.curr_state}.npy',lidar)
            steering_angle,speed = self.angles[angle_index],self.speeds[speed_index]
            curr_x = obs['poses_x'][0]
            curr_y = obs['poses_y'][0]
            obs, step_reward, done, info = self.env.step(np.array([[steering_angle, speed]]))
            lidar = obs['scans'][0]
            lidar_down_sampled = self.get_statistical_properties(lidar,n_sectors=self.n_sectors)
            self.next_state = self.get_state(self.get_binary_representation(lidar_down_sampled))

            if done:
                self.reward =-1000
            else:           
                # self.reward = self.reward_class.calculate_reward(np.array([curr_x, curr_y]), np.array([obs['poses_x'][0], obs['poses_y'][0]]),obs['ang_vels_z'][0])
                self.reward = self.reward_class.calculate_reward(np.array([curr_x, curr_y]), np.array([obs['poses_x'][0], obs['poses_y'][0]]),self.speeds[speed_index])
            
            self.episode_reward += self.reward

            self.set_eligibility_traces(angle_index,speed_index)

            if done:
                self.track_center_counter-=1
                self.logger.info(f'Collision: {self.num_collisions+1}, State: {self.curr_state}, Action: {angle_index,speed_index}, Reward: {self.episode_reward}')
                self.action_threshold *= self.action_threshold_decay
                self.episodic_rewards.append(self.episode_reward)
                self.episode_reward = 0
                end_time = time.time()
                self.collision_times.append(end_time - start_time)
                start_time = end_time

                self.num_collisions += 1

                self.ET_IS.fill(0)
  

                # Obtaining a new random position on the track
                random_idx = self.index_selector.select_index()
                n_x, n_y = self.map_centers[random_idx]
                delta_x, delta_y = np.random.uniform(-0.75, 0.75), np.random.uniform(-0.3, 0.3)
                delta_theta = np.random.uniform(-0.2, 0.2)
                n_theta = self.track_headings[random_idx]

                # Sensing the new state
                obs, step_reward, done, info = self.env.reset(np.array([[n_x + delta_x, n_y + delta_y, n_theta+delta_theta]]))
                lidar = obs['scans'][0]
                lidar_down_sampled = self.get_statistical_properties(lidar,n_sectors=self.n_sectors)
                self.curr_state = self.get_state(self.get_binary_representation(lidar_down_sampled))
                angle_index,speed_index = self.select_action(self.curr_state)

                # Resetting 
                self.reward_class.reset(np.array([[n_x + delta_x, n_y + delta_y]]))
                
                # Checkpoint
                if (self.num_collisions+1) % 500 == 0:
                    print(f'Collision: {self.num_collisions+1}, Time: {sum(self.collision_times)}, Reward: {sum(self.episodic_rewards)/len(self.episodic_rewards)}')
                    self.save_reward_time()
                    self.save_weights()
                    self.episodic_rewards.clear()
                    self.collision_times.clear()
                    self.reward_file = os.path.join(self.save_path, f'rewards.npy')
                    self.collision_file = os.path.join(self.save_path, f'times.npy')
                    
                if  self.track_center_counter == 0:
                    print(f'Training on {self.track_name[self.path_counter]} Completed')
                    self.__update_map()
                    self.reward_class.set_parameters(self.map_centers,self.track_width)
                    self.reward_class.reset(np.array([[self.sx, self.sy]]))
                    self.env.reset(np.array([[self.sx, self.sy, self.track_headings[0]]]))
                    self.index_selector.set_parameters(self.map_centers.shape[0])
            

            angle_index,speed_index = self.sarsa_weight_update(angle_index,speed_index,self.reward)
            self.decay_eligibility_traces()
            self.curr_state = self.next_state
            # self.env.render(mode='human')

    def inference(self):
        obs, step_reward, done, info = self.env.reset(np.array([[self.sx, self.sy, self.track_headings[0]]]))
        lidar = obs['scans'][0]
        lidar_down_sampled = self.get_statistical_properties(lidar,n_sectors=self.n_sectors)
        self.curr_state = self.get_state(self.get_binary_representation(lidar_down_sampled))
        self.reward_class.reset(np.array([[self.sx, self.sy]]))
        angle_index,speed_index = self.select_action_inference(self.curr_state)
        while not done:
            steering_angle,speed = self.angles[angle_index],self.speeds[speed_index]
            obs, step_reward, done, info = self.env.step(np.array([[steering_angle, speed]]))
            lidar = obs['scans'][0]
            lidar_down_sampled = self.get_statistical_properties(lidar,n_sectors=self.n_sectors)
            self.next_state = self.get_state(self.get_binary_representation(lidar_down_sampled))
            angle_index,speed_index = self.select_action_inference(self.next_state)
            self.curr_state = self.next_state
            
            self.env.render(mode='human')

In [ ]:
path = './f1tenth_racetracks'
all_map_paths=[]
map_centers = []
map_names = []
track_lengths=[]
for folder in os.listdir(path):
    if folder not in ['README.md','.gitignore','convert.py','LICENSE','rename.py','.git']:
        folder_name=folder
        file_name=folder_name.replace(' ','')+'_map'
        map_center = folder_name.replace(' ','')+'_centerline.csv'
        track_lengths.append(len(pd.read_csv(f'{path}/{folder_name}/{map_center}')))
        map_names.append(folder_name)
        all_map_paths.append(f'{path}/{folder_name}/{file_name}')
        map_centers.append(f'{path}/{folder_name}/{map_center}')

track_length_list = list(zip(map_names,track_lengths))
track_length_list

In [ ]:
train_maps = ['Spa','Hockenheim','Shanghai','Nuerburgring','Montreal', 'Austin','Mexico City']
test_maps = [i[0] for i in track_length_list if i[0] not in train_maps]
print(f'Train Maps: {train_maps}')
print(f'Test Maps: {test_maps}')

In [ ]:
global num_agents,map_path,map_ext,sx,sy,stheta,num_maps_to_train,gym_env_code,inference_file,reward_file,collision_file,counter,indices,save_path
gym_env_code='f110_gym:f110-v0'
num_agents = 1
map_ext = '.png'
sx = 0.
sy = 0.
stheta = {'Hockenheim': 2.02,
 'Mexico City': -0.15,
 'Oschersleben': 2.86,
 'Shanghai': -2.93,
 'BrandsHatch': 0.42,
 'Monza': 1.47,
 'Catalunya': -2.14,
 'SaoPaulo': -1.31,
 'Sepang': -3.06,
 'Silverstone': 0.94,
 'Nuerburgring': -2.38,
 'YasMarina': 0.13,
 'Spa': 2.13,
 'Sochi': -2.14,
 'Montreal': -1.35,
 'Austin': -0.65,
 'Melbourne': 2.37,
 'Budapest': 2.45,
 'Spielberg': -2.88,
 'Zandvoort': 1.2,
 'Sakhir': 1.53,
 'MoscowRaceway': 1.46}

indices = [idx for idx,i in enumerate(map_names) if i in train_maps]
save_path = 'SARSA_Multiple_binary_training/'
if not os.path.exists(save_path):
    os.mkdir(save_path)
counter = 0
# inference_file = 'SARSA_Fewer_Multiple_training/Mexico City_34500.npy'
# reward_file='SARSA_Fewer_Multiple_training/rewards.npy'
# collision_file='SARSA_Fewer_Multiple_training/times.npy'
inference_file = None
reward_file = None
collision_file = None
map_path_subset = [all_map_paths[i] for i in indices]
map_centers_subset = [map_centers[i] for i in indices]
map_names_subset = [map_names[i] for i in indices]

simulator = F1Tenth_navigation(gym_env_code=gym_env_code, num_agents=num_agents, map_path=map_path_subset, map_ext=map_ext, sx=sx, sy=sy, stheta=stheta, map_centers_file=map_centers_subset, save_path=save_path, track_name=map_names_subset, inference=inference_file,reward_file=reward_file,collision_file=collision_file)
simulator.train()